In [1]:
import pandas as pd
import numpy as np
import csv
from tqdm import tqdm
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

d = datetime.today().strftime('%Y%m%d-%H%M%S')
data_dir = '/work/users/k/4/k4thryn/Repos/EpSampling/data'

In [2]:
def time_to_int(date):
    y,m,d = date.split('-')
    print(y,m,d)
    return 10*y + 100*m + d
# time_to_int('2022-05-07')

In [3]:
import glob
def get_latest(csv_name, data_dir=f'{data_dir}/processed/'):
#     csv_name = 'all_county_acs_covs'
    dates = sorted([x[-19:-4] for x in glob.glob(f'{data_dir}{csv_name}*')])
    latest = dates[-1]
    file_name = f'{data_dir}{csv_name}_{latest}.csv'
    return file_name

In [5]:
# print(get_latest('all_county_acs_covs',f'{data_dir}/processed/'))

csv = get_latest('merged_covs_deaths',f'{data_dir}/processed/')
_df = pd.read_csv(csv)

acs_covs = list(_df.columns[11:].values)
death_covs = ['Fips','Date','COVIDhubEns_state_deaths', 'Pop', 'Pop_ratio','Naive_county_deaths']
target = ['True_county_deaths']

all_covs = target + death_covs + acs_covs
df = _df[all_covs]

In [7]:
len(df)
df.head()

,True_county_deaths,Fips,Date,COVIDhubEns_state_deaths,Pop,Pop_ratio,Naive_county_deaths,POP_x2,POP_M,POP_F,...,HU_x15,HU_UIS01D,HU_UIS01A,HU_UIS02,HU_UIS0304,HU_UIS0509,HU_UIS1019,HU_UIS2049,HU_UIS50P,HU_UISOTHER
0,0.0,41001,2020-05-09,125.655,16124,0.003823,0.480367,16539,8458,8081,...,8654,6300,109,162,159,183,105,90,157,1389
1,0.0,41001,2020-05-16,143.950,16124,0.003823,0.550307,16539,8458,8081,...,8654,6300,109,162,159,183,105,90,157,1389
2,0.0,41001,2020-05-23,150.378,16124,0.003823,0.574881,16539,8458,8081,...,8654,6300,109,162,159,183,105,90,157,1389
3,0.0,41001,2020-05-30,159.093,16124,0.003823,0.608197,16539,8458,8081,...,8654,6300,109,162,159,183,105,90,157,1389
4,0.0,41001,2020-06-06,164.199,16124,0.003823,0.627717,16539,8458,8081,...,8654,6300,109,162,159,183,105,90,157,1389


In [ ]:
from sklearn.model_selection import train_test_split


df_train, df_test = train_test_split(df, test_size=0.2, random_state=666)

target = 'True_county_deaths'

covs_to_drop = [target,
#                 'Fips',
#                 'Naive_county_deaths',
#                 'Pred_state_deaths'
               ]

X_train = df_train.drop(covs_to_drop,axis=1)
y_train = df_train[target].values

X_test = df_test.drop(covs_to_drop,axis=1)
y_test = df_test[target].values

y_naive = df_test['Naive_county_deaths'].values

print(len(X_train), len(X_test))


from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(random_state=666)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test) #[:,None]
y_prob = clf.predict_proba(X_test)[:,0]

# y_naive = X_test['Naive_county_deaths'].values

# print(y_test[:10])
# print(y_pred[:10])
# print(y_naive[:10])





265949 66488


In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

for pred in [y_naive, y_pred]:
    mae = mean_absolute_error(y_test, pred)
    mse = mean_squared_error(y_test, pred)
    r2 = r2_score(y_test, pred)
    print(mae, mse, r2)

In [ ]:
df.columns[:15]

In [ ]:
# # from IPython.display import Audio, display
# import IPython

# def allDone():
# #     urL = 'https://www.wavsource.com/snds_2020-10-01_3728627494378403/animals/cat_meow2.wav'
# #     urL = 'https://www.wavsource.com/snds_2020-10-01_3728627494378403/animals/cat_meow2.wav'
# #     display(Audio(url=urL, autoplay=True))
    
#     IPython.display.Audio('../cat_meow2.wav',autoplay=True)
# allDone()